# Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from tqdm.auto import tqdm
import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)
cudnn.benchmark = True

 ###### Grab a GPU if there is one 

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using {} device: {}".format(device, torch.cuda.current_device()))
else:
    device = torch.device("cpu")
    print("Using {}".format(device))

Using cuda device: 0


#### Load data

In [3]:
data = np.load('capture24.npz', allow_pickle=True)
print("Contents of capture24.npz:", data.files)
X_feats, y, pid, time, annotation = \
    data['X_feats'], data['y'], data['pid'], data['time'], data['annotation']
print('X_feats shape:', X_feats.shape)
print('y shape:', y.shape)
print('pid shape:', pid.shape)
print('time shape:', time.shape)
print('annotation shape:', annotation.shape)
X_raw = np.load('X_raw.npy', mmap_mode='r')
print('X_raw shape:', X_raw.shape)
data_unl = np.load('capture24_test.npz')
print("\nContents of capture24_test.npz:", data_unl.files)

Contents of capture24.npz: ['X_feats', 'y', 'pid', 'time', 'annotation']
X_feats shape: (330610, 125)
y shape: (330610,)
pid shape: (330610,)
time shape: (330610,)
annotation shape: (330610,)
X_raw shape: (330610, 3, 3000)

Contents of capture24_test.npz: ['X_feats', 'pid', 'time']


In [4]:
def interval2seq(mydata):
    X_feats, y, pid, time, annotation = \
    data['X_feats'], data['y'], data['pid'], data['time'], data['annotation']
    
    # Get all the unique pids 
    subjects = np.unique(data['pid'])
    X_tr = []
    y_tr =  []
    pid_tr =  []
    for subject in subjects:
        X_tr.append(X_raw[pid==subject])
        y_tr.append(y[pid==subject])
        pid_tr.append(subject)
    x = np.array(X_tr)
    y = np.array(y_tr)
    pid = np.array(pid_tr)
    
    return x, y, pid

In [5]:
x, y, pid = interval2seq(data)

# Split the train&test 

In [6]:
# Hold out some participants for testing the model
pids_test = [2, 3]
mask_test = np.isin(pid, pids_test)
mask_train = ~mask_test
y_train, y_test = y[mask_train], y[mask_test]
pid_train, pid_test = pid[mask_train], pid[mask_test]
# X[mask_train] and X[mask_test] if you like to live dangerously
X_train = utils.ArrayFromMask(x, mask_train)
X_test = utils.ArrayFromMask(x, mask_test)
print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)

Shape of X_train (137,)
Shape of X_test (2,)


In [19]:
X_train[0].shape

(977, 3, 3000)

In [20]:
X_train[1].shape

(2051, 3, 3000)

### Architecture design

In [10]:
class ConvBNReLU(nn.Module):
    ''' Convolution + batch normalization + ReLU is a common trio '''
    def __init__(
        self, in_channels, out_channels,
        kernel_size=5, stride=1, padding=1, bias=True
    ):
        super(ConvBNReLU, self).__init__()

        self.main = nn.Sequential(
            nn.Conv1d(in_channels, out_channels,
                kernel_size, stride, padding, bias=bias),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(True)
        )

    def forward(self, x):
        return self.main(x)


class CNN(nn.Module):
    ''' Typical CNN design with pyramid-like structure '''
    def __init__(self, output_size=5, in_channels=3, num_filters_init=8):
        super(CNN, self).__init__()

        self.cnn = nn.Sequential(
            ConvBNReLU(in_channels, num_filters_init,
            8, 4, 2, bias=False),  # 1500 -> 750
            ConvBNReLU(num_filters_init, num_filters_init*2,
            6, 4, 2, bias=False),  # 750 -> 188
            ConvBNReLU(num_filters_init*2, num_filters_init*4,
            8, 4, 2, bias=False),  # 188 -> 47
            ConvBNReLU(num_filters_init*4, num_filters_init*8,
            3, 2, 1, bias=False),  # 47 -> 24
            ConvBNReLU(num_filters_init*8, num_filters_init*16,
            4, 2, 1, bias=False),  # 24 -> 12
            ConvBNReLU(num_filters_init*16, num_filters_init*32,
            4, 2, 1, bias=False),  # 12 -> 6
            ConvBNReLU(num_filters_init*32, num_filters_init*64,
            6, 1, 0, bias=False),  # 6 -> 1
            nn.Conv1d(num_filters_init*64, output_size,
            1, 1, 0, bias=True)
        )

    def forward(self, x):
        return self.cnn(x).view(x.shape[0],-1)

In [11]:
class CNN(nn.Module):
    ''' Typical CNN design with pyramid-like structure '''
    def __init__(self, output_size=128, in_channels=3, num_filters_init=8):
        super(CNN, self).__init__()

        self.cnn = nn.Sequential(
            ConvBNReLU(in_channels, 64),
            ConvBNReLU(64, 64),
            ConvBNReLU(64, 64),
            ConvBNReLU(64, 64)
        )

    def forward(self, x):
        return self.cnn(x).view(x.shape[0],-1)

In [12]:
class Combine(nn.Module):
    def __init__(self, num_hn=128, num_class=5):
        super(Combine, self).__init__()
        self.cnn = CNN()
        cnnoutputsize = 234
        self.rnn1 = nn.LSTM(
            input_size=cnnoutputsize, 
            hidden_size=num_hn, 
            batch_first=True)
        self.rnn1 = nn.LSTM(
            input_size=num_hn, 
            hidden_size=num_hn, 
            batch_first=True)
        self.linear = nn.Linear(num_hn, num_class)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = self.cnn(c_in)
        
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out, (h_n, h_c) = self.rnn1(r_in)
        r_out, (h_n, h_c) = self.rnn1(r_in)

        r_out2 = self.linear(r_out[:, -1, :])
        
        return F.log_softmax(r_out2, dim=1)

In [14]:
num_filters_init = 8  # initial num of filters -- see class definition
in_channels = 3  # num channels of the signal -- equal to 3 for our raw triaxial timeseries
output_size = 5  # number of classes (sleep, sedentary, etc...)
num_epoch = 5  # num epochs (full loops though the training set) for SGD training
lr = 1e-3  # learning rate in SGD
batch_size = 32  # size of the mini-batch in SGD

#cnn = CNN(
#    output_size=output_size,
#    in_channels=in_channels,
#).to(device)

cnnlstm = Combine().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnnlstm.parameters(), lr=lr, amsgrad=True)
print(cnnlstm)

Combine(
  (cnn): CNN(
    (cnn): Sequential(
      (0): ConvBNReLU(
        (main): Sequential(
          (0): Conv1d(3, 64, kernel_size=(5,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (1): ConvBNReLU(
        (main): Sequential(
          (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (2): ConvBNReLU(
        (main): Sequential(
          (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (3): ConvBNReLU(
        (main): Sequential(
          (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(1,))
          

In [15]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        print(nn)
        pp += nn
    return pp

# Feed the data into the network

In [17]:
def create_dataloader(X, y=None, batch_size=1, shuffle=False):
    ''' Create a (batch) iterator over the dataset. Alternatively, use PyTorch's
    Dataset and DataLoader classes -- See
    https://pytorch.org/tutorials/beginner/data_loading_tutorial.html '''
    if shuffle:
        idxs = np.random.permutation(np.arange(len(X)))
    else:
        idxs = np.arange(len(X))
    for i in range(0, len(idxs), batch_size):
        idxs_batch = idxs[i:i+batch_size]
        X_batch = X[idxs_batch]
        print(X_batch.shape)
        X_batch = torch.from_numpy(X_batch)
        if y is None:
            yield X_batch
        else:
            y_batch = y[idxs_batch]
            y_batch = torch.from_numpy(y_batch)
            yield X_batch, y_batch


def forward_by_batches(cnn, X):
    ''' Forward pass model on a dataset. Do this by batches so that we do
    not blow up the memory. '''
    Y = []
    cnn.eval()
    with torch.no_grad():
        for x in create_dataloader(X, batch_size=1024, shuffle=False):  # do not shuffle here!
            x = x.to(device)
            Y.append(cnn(x))
    cnn.train()
    Y = torch.cat(Y)
    return Y

def evaluate_model(cnn, prior, emission, transition, X, y, pid=None):
    Y_pred = forward_by_batches(cnn, X)  # scores
    loss = F.cross_entropy(Y_pred, torch.from_numpy(y).to(device)).item()
    Y_pred = F.softmax(Y_pred, dim=1)  # convert to probabilities
    y_pred = torch.argmax(Y_pred, dim=1)  # convert to classes
    y_pred = y_pred.cpu().numpy()  # cast to numpy array
    y_pred = utils.viterbi(y_pred, prior, transition, emission)  # HMM smoothing
    scores = utils.compute_scores(y, y_pred)
    return loss, scores

In [18]:
accuracy_history = []
balanced_accuracy_history = []
kappa_history = []
loss_history = []
loss_history_train = []
for i in tqdm(range(num_epoch)):
    dataloader = create_dataloader(X_train, y_train, batch_size, shuffle=True)
    losses = []
    for x, target in dataloader:
        x, target = x.to(device), target.to(device)
        cnn.zero_grad()
        output = cnnlstm(x)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        # Logging -- track train loss
        losses.append(loss.item())

    # -------------------------------------------------------------------------
    # Evaluate performance at the end of each epoch (full loop through the
    # training set). We could also do this at every iteration, but this would
    # be very expensive since we are evaluating on a large dataset.
    # Aditionally, at the end of each epoch we train a Hidden Markov Model to
    # smooth the predictions of the CNN.
    # -------------------------------------------------------------------------

    # Logging -- average train loss in this epoch
    loss_history_train.append(np.mean(losses))

    # Compute HMM params
    prior, emission, transition = train_hmm(cnn, X_train, y_train)

    # Logging -- evalutate performance on test set
    loss_test, scores_test = evaluate_model(
        cnn, prior, emission, transition, X_test, y_test, pid_test
    )
    loss_history.append(loss_test)
    accuracy_history.append(scores_test['accuracy'])
    balanced_accuracy_history.append(scores_test['balanced_accuracy'])
    kappa_history.append(scores_test['kappa'])

(32,)



TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.